# Feature Selection with emission estimation

Feature selection, or *variable selection*, is an often used technique in machine learning. It is a process of selecting subset of highly relevant data to benefit modelling in many ways. Such as,

- Reduce the complexity, that is, improve the efficiency of training [1].

- Improve the prediction accuracy [1,2].




## Find the top *k* features most correlated to Saleprice (Naïve method)


### Data Description

(House price data train.csv from https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data)

The data have 1460 rows and 81 columns. The following code will print out the data description in detail.

In [36]:
try:
    with open("data/data_description.txt", 'r') as file:
        # Read the content of the file
        file_content = file.read()
        
        # Print the content
        print(f"{file_content}\n")

except FileNotFoundError:
    print(f"File '{file_path}' not found.")
except Exception as e:
    print(f"An error occurred: {e}")

MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM

### Load required libraries

In [37]:
import pandas as pd
import numpy as np

import time
import psutil
import os,gc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
#from codecarbon import EmissionsTracker
from codecarbon import OfflineEmissionsTracker


### Define functions for data preprocessing

In [38]:

def get_memory_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024  # Memory in MB

def naive_feature_selection(X, y, k=20):
    #mem_use_before = get_memory_usage()
    #print(f"Memory before naive selection: {mem_use_before:.2f} MB")

    # Select only numerical columns and fill missing values with median values of the columns
    X_numeric = X.select_dtypes(include=[np.number]).fillna(X.select_dtypes(include=[np.number]).median())
    # Compute correlations for numerical features only
    n_features = X_numeric.shape[1]
    correlations = np.zeros(n_features)
    for i in range(n_features):
        valid = (~np.isnan(X_numeric.iloc[:, i]) & ~np.isnan(y))
        if valid.sum() > 1:
            correlations[i] = np.abs(np.corrcoef(X_numeric.iloc[:, i][valid], y[valid])[0, 1])
        else:
            correlations[i] = 0
    top_k_indices = np.argsort(correlations)[-k:]
    selected_features = X_numeric.columns[top_k_indices].tolist()
    #mem_use_after = get_memory_usage()
    #print(f"Memory after naive selection: {mem_use_after:.2f} MB")
    #print(f"memory use increased {(mem_use_after - mem_use_before):.2f} MB" )
    return selected_features

def load_and_evaluate(file_path, k=20, n_runs = 100):
    # Initialize lists to store metrics
    memory_before_list = []
    memory_after_list = []
    memory_increase_list = []
    runtime_list = []
    mse_list = []
    emissions_list = []

    #load data
    data = pd.read_csv(file_path)
    # split Sale price and other columns
    X = data.drop(columns=['SalePrice', 'Id'])
    y = data['SalePrice']
    # Handle missing values for numerical columns 
    X_numeric = X.select_dtypes(include=[np.number]).fillna(X.select_dtypes(include=[np.number]).median())
    X_categorical = X.select_dtypes(exclude=[np.number])
    # Combine numerical and categorical columns
    X = pd.concat([X_numeric, X_categorical], axis=1)
    
    # Verify no NaNs remain 
    if X.isna().any().any():
        X = X.fillna(0)  # Fill any remaining NaNs with 0 (e.g., for edge cases)
    
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Monte Carlo method for testing mem use and runtime
    print(f"Running naive method {n_runs} times...")
    # Calling psutil.cpu_precent() for 1 seconds
    cpu_use_start = psutil.cpu_percent(1)

    #tracker = EmissionsTracker(project_name="naive_feature_selection", output_file="naive_emissions.csv")
    tracker = OfflineEmissionsTracker(
        project_name="naive_feature_selection",
        output_file="naive_emissions.csv",
        country_iso_code="GBR",
        log_level="warning"
    )
    #tracker = OfflineEmissionsTracker(country_iso_code="UK")
    tracker.start()

    for run in range(n_runs):
        # clear memory before each run
        gc.collect() 
        mem_use_before = get_memory_usage()
        #print(f"Memory before naive selection: {mem_use_before:.2f} MB")

        start_time = time.time()
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42+run)
        selected_features = naive_feature_selection(X_train, y_train, k)
        X_train_selected = X_train[selected_features]
        X_test_selected = X_test[selected_features]
        model = LinearRegression()
        model.fit(X_train_selected, y_train)
        y_pred = model.predict(X_test_selected)
        mse = mean_squared_error(y_test, y_pred)

        mem_use_after = get_memory_usage()
        end_time = time.time()
        # Store metrics
        memory_before_list.append(mem_use_before)
        memory_after_list.append(mem_use_after)
        memory_increase_list.append(mem_use_after - mem_use_before)
        runtime_list.append(end_time - start_time)
        mse_list.append(mse)

    emissions = tracker.stop()  # Emissions in kg CO₂eq
    cpu_use_end = psutil.cpu_percent(1)
    scaled_emissions = emissions * (cpu_use_end - cpu_use_start) / 100
    emissions_list.append(scaled_emissions)
    #print(f"Memory after naive selection: {mem_use_after:.2f} MB")

    #print(f"Naive feature selection took {time.time() - start_time} seconds")
    
    # Ensure selected features are NaN-free
    #X_train_selected = X_train[selected_features]
    #X_test_selected = X_test[selected_features]
    
    #model = LinearRegression()
    #model.fit(X_train_selected, y_train)
    #y_pred = model.predict(X_test_selected)
    #mse = mean_squared_error(y_test, y_pred)
    #print(f"Mean Squared Error with {k} features: {mse}")

    # Compute statistics
    stats = {
        'memory_before_mean (MB)': np.mean(memory_before_list),
        'memory_before_std': np.std(memory_before_list),
        'memory_after_mean (MB)': np.mean(memory_after_list),
        'memory_after_std': np.std(memory_after_list),
        'memory_increase_mean (MB)': np.mean(memory_increase_list),
        'memory_increase_std': np.std(memory_increase_list),
        'runtime_mean (Seconds)': np.mean(runtime_list),
        'runtime_std': np.std(runtime_list),
        'MSE_mean': np.mean(mse_list),
        'MSE_std': np.std(mse_list),
        'emissions_total (kg CO2eq)': np.sum(emissions_list)

    }
    
    return selected_features, mse, stats

def generate_report(stats, method="Naive"):
    report = f"# {method} Method Statistical Report\n\n"
    report += "## Summary Statistics\n\n"
    report += "| Metric | Mean | Standard Deviation |\n"
    report += "|--------|------|--------------------|\n"
    report += f"| Memory Before (MB) | {stats['memory_before_mean (MB)']:.2f} | {stats['memory_before_std']:.2f} |\n"
    report += f"| Memory After (MB) | {stats['memory_after_mean (MB)']:.2f} | {stats['memory_after_std']:.2f} |\n"
    report += f"| Memory Increase (MB) | {stats['memory_increase_mean (MB)']:.2f} | {stats['memory_increase_std']:.2f} |\n"
    report += f"| Runtime (seconds) | {stats['runtime_mean (Seconds)']:.6f} | {stats['runtime_std']:.6f} |\n"
    report += f"| MSE | {stats['MSE_mean']:.2f} | {stats['MSE_std']:.2f} |\n"
    report += f"\n## Environmental Impact\n"
    report += f"- Total Emissions (kg CO2eq): {stats['emissions_total (kg CO2eq)']:.6f}\n"

    return report

### Main process of evaluation with Monte Carlo

In [39]:

if __name__ == "__main__":
    gc.collect()
    file_path = "data/train.csv"
    n_runs = 500
    selected_features, mse, stats = load_and_evaluate(file_path,  k=20, n_runs=n_runs)
    #print(f"Selected features (last run): {selected_features}")
    report = generate_report(stats, method="Naive")
    with open("naive_stats_report.md", "w") as f:
        f.write(report)
    print("Naive method report generated: naive_stats_report.md")
    print("\nSummary Statistics:")
    for key, value in stats.items():
        print(f"{key}: {value:.6f}")
    #mem_use_after = get_memory_usage()
    #print(f"Memory after naive selection: {mem_use_after:.2f} MB")
    #print(f"memory use increased {(mem_use_after - mem_use_before):.2f} MB" )
    print(f"Selected features: {selected_features}")

Running naive method 500 times...


[codecarbon WARNING @ 17:18:06] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon WARNING @ 17:18:06] Error while trying to count physical CPUs: [Errno 2] No such file or directory: 'lscpu'. Defaulting to 1.
[codecarbon WARNING @ 17:18:07] We saw that you have a Apple M4 but we don't know it. Please contact us.
[codecarbon WARNING @ 17:18:13] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Mac OS detected: Please install Intel Power Gadget or enable PowerMetrics sudo to measure CPU

[codecarbon WARNING @ 17:18:13] No CPU tracking mode found. Falling back on CPU constant mode.
/usr/local/lib/python3.12/site-packages/codecarbon/output_methods/file.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the 

Naive method report generated: naive_stats_report.md

Summary Statistics:
memory_before_mean (MB): 157.774633
memory_before_std: 2.341800
memory_after_mean (MB): 157.815586
memory_after_std: 2.302056
memory_increase_mean (MB): 0.040953
memory_increase_std: 0.194648
runtime_mean (Seconds): 0.021939
runtime_std: 0.006123
MSE_mean: 1495907264.707669
MSE_std: 668916768.381671
emissions_total (kg CO2eq): 0.000012
Selected features: ['LotArea', 'HalfBath', '2ndFlrSF', 'WoodDeckSF', 'LotFrontage', 'OpenPorchSF', 'BsmtFinSF1', 'Fireplaces', 'MasVnrArea', 'GarageYrBlt', 'YearRemodAdd', 'YearBuilt', 'TotRmsAbvGrd', 'FullBath', '1stFlrSF', 'TotalBsmtSF', 'GarageArea', 'GarageCars', 'GrLivArea', 'OverallQual']


In [40]:
!python3 -m pip install --no-binary :all: psutil


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python3.12 -m pip install --upgrade pip


## Reference

1. Gareth James; Daniela Witten; Trevor Hastie; Robert Tibshirani (2013). "An Introduction to Statistical Learning". *Springer*. p. 204.
2. Kratsios, Anastasis; Hyndman, Cody (2021). "NEU: A Meta-Algorithm for Universal UAP-Invariant Feature Representation". *Journal of Machine Learning Research*. 22 (92): 1–51